# Tests extracción de rasgos estilométricos con RAID

In [14]:
# import numpy as np
# import math
# import random
# import matplotlib.pyplot as plt
# import cvxopt
# import seaborn as sns
# import soundfile as sf
# import itertools
# import pywt
# import librosa
# import librosa.display
# from IPython.display import Audio
# from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_score, recall_score, f1_score
# from cvxopt import matrix, solvers
# import pandas as pd
# import os
# import scipy
# import scipy.signal
# from sklearn.decomposition import PCA, TruncatedSVD, FactorAnalysis
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.model_selection import train_test_split
# from sklearn.random_projection import SparseRandomProjection, johnson_lindenstrauss_min_dim, GaussianRandomProjection
# from sklearn.svm import SVC
# from sklearn.preprocessing import StandardScaler
# import torch
# from torch import nn
# import torch.optim as optim
# import torch.nn.functional as F
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression

In [15]:
import stylo_metrix as sm
import pandas as pd
from raid import run_detection, run_evaluation
from raid.utils import load_data

from processer import split_text_into_sentences

## Manejo de RAID

In [16]:
# Download the RAID dataset without adversarial attacks
or_train_noadv_df = load_data(split="train", include_adversarial=False)
# test_noadv_df = load_data(split="test", include_adversarial=False)
# extra_noadv_df = load_data(split="extra", include_adversarial=False)

In [17]:
display(or_train_noadv_df.head(7))

,id,adv_source_id,source_id,model,decoding,repetition_penalty,attack,domain,title,prompt,generation
0,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,human,NaN,NaN,none,abstracts,FUTURE-AI: Guiding Principles and Consensus Re...,NaN,The recent advancements in artificial intellig...
1,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,human,NaN,NaN,none,abstracts,EdgeFlow: Achieving Practical Interactive Segm...,NaN,High-quality training data play a key role in ...
2,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,human,NaN,NaN,none,abstracts,Semi-supervised Contrastive Learning for Label...,NaN,The success of deep learning methods in medica...
3,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,human,NaN,NaN,none,abstracts,Combo Loss: Handling Input and Output Imbalanc...,NaN,Simultaneous segmentation of multiple organs f...
4,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,human,NaN,NaN,none,abstracts,Attention-Based 3D Seismic Fault Segmentation ...,NaN,Detection faults in seismic data is a crucial ...
5,72fe360b-cce6-4daf-b66a-1d778f5964f8,72fe360b-cce6-4daf-b66a-1d778f5964f8,72fe360b-cce6-4daf-b66a-1d778f5964f8,human,NaN,NaN,none,abstracts,Segmenter: Transformer for Semantic Segmentation,NaN,Image segmentation is often ambiguous at the l...
6,df594cf4-9a0c-4488-bcb3-68f41e2d5a16,df594cf4-9a0c-4488-bcb3-68f41e2d5a16,df594cf4-9a0c-4488-bcb3-68f41e2d5a16,human,NaN,NaN,none,abstracts,Mining Contextual Information Beyond Image for...,NaN,This paper studies the context aggregation pro...


In [18]:
intCols = ['id','model', 'domain', 'title', 'prompt', 'generation']
# print("Visualizar columnas específicas:")
# train_noadv_df = or_train_noadv_df[or_train_noadv_df['model'] != 'human']

# Copia del dataframe con columnas específicas
train_noadv_df = or_train_noadv_df.copy()
train_noadv_df = train_noadv_df[intCols]

display(train_noadv_df.head(7))

,id,model,domain,title,prompt,generation
0,e5e058ce-be2b-459d-af36-32532aaba5ff,human,abstracts,FUTURE-AI: Guiding Principles and Consensus Re...,NaN,The recent advancements in artificial intellig...
1,f95b107b-d176-4af5-90f7-4d0bb20caf93,human,abstracts,EdgeFlow: Achieving Practical Interactive Segm...,NaN,High-quality training data play a key role in ...
2,856d8972-9e3d-4544-babc-0fe16f21e04d,human,abstracts,Semi-supervised Contrastive Learning for Label...,NaN,The success of deep learning methods in medica...
3,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,human,abstracts,Combo Loss: Handling Input and Output Imbalanc...,NaN,Simultaneous segmentation of multiple organs f...
4,72c41b8d-0069-4886-b734-a4000ffca286,human,abstracts,Attention-Based 3D Seismic Fault Segmentation ...,NaN,Detection faults in seismic data is a crucial ...
5,72fe360b-cce6-4daf-b66a-1d778f5964f8,human,abstracts,Segmenter: Transformer for Semantic Segmentation,NaN,Image segmentation is often ambiguous at the l...
6,df594cf4-9a0c-4488-bcb3-68f41e2d5a16,human,abstracts,Mining Contextual Information Beyond Image for...,NaN,This paper studies the context aggregation pro...


In [19]:
print("Información del dataset de entrenamiento:")
print(f"Forma del dataset: {train_noadv_df.shape}")
print(f"Columnas: {list(train_noadv_df.columns)}")
print(f"Modelos unicos: {train_noadv_df['model'].unique()}")
print(f"Dominios unicos: {train_noadv_df['domain'].unique()}")

Información del dataset de entrenamiento:
Forma del dataset: (467985, 6)
Columnas: ['id', 'model', 'domain', 'title', 'prompt', 'generation']
Modelos unicos: ['human' 'llama-chat' 'mpt' 'mpt-chat' 'gpt2' 'mistral' 'mistral-chat'
 'gpt3' 'cohere' 'chatgpt' 'gpt4' 'cohere-chat']
Dominios unicos: ['abstracts' 'books' 'news' 'poetry' 'recipes' 'reddit' 'reviews' 'wiki']


In [20]:
# intCols = ['model', 'domain', 'title', 'prompt', 'generation']
# print("Visualizar columnas específicas:")
filtered = train_noadv_df[train_noadv_df['model'] != 'human']

display(filtered.head(7))

,id,model,domain,title,prompt,generation
493,2bd98bd7-3356-43bf-8c5d-69ef336d0536,llama-chat,abstracts,FUTURE-AI: Guiding Principles and Consensus Re...,Write the abstract for the academic paper titl...,"In the paper ""FUTURE-AI: Guiding Principles an..."
494,e8bdc461-3ff2-4d68-8c7b-cdbc086f62b3,llama-chat,abstracts,FUTURE-AI: Guiding Principles and Consensus Re...,Write the abstract for the academic paper titl...,"In the paper ""Future-AI: Guiding Principles an..."
495,ee968d29-ce73-4c5d-804d-0a0efec4bea4,llama-chat,abstracts,EdgeFlow: Achieving Practical Interactive Segm...,Write the abstract for the academic paper titl...,"In this paper, we present EdgeFlow, a novel ap..."
496,3d24eb90-f540-490f-81c8-e4a24fd49ad7,llama-chat,abstracts,EdgeFlow: Achieving Practical Interactive Segm...,Write the abstract for the academic paper titl...,"In this paper, we present a novel approach to ..."
497,5f817bbe-4fb4-4011-a1e9-fcf12990f450,llama-chat,abstracts,Semi-supervised Contrastive Learning for Label...,Write the abstract for the academic paper titl...,"In this paper, we propose a novel approach to ..."
498,7389e65b-2e27-4b90-999a-53e28b773315,llama-chat,abstracts,Semi-supervised Contrastive Learning for Label...,Write the abstract for the academic paper titl...,"In this paper, we propose a novel approach to ..."
499,8b79a378-67db-48e8-8950-4d3215cfef16,llama-chat,abstracts,Combo Loss: Handling Input and Output Imbalanc...,Write the abstract for the academic paper titl...,"In the field of medical image segmentation, im..."


In [21]:
# Opción 1: Configurar pandas para mostrar más contenido
pd.set_option('display.max_colwidth', None)  # Sin límite de ancho
pd.set_option('display.max_rows', None)      # Sin límite de filas (usar con cuidado)

complex_filter = train_noadv_df[(train_noadv_df['model'] == 'human') &
                                (train_noadv_df['domain'] == 'abstracts')]

# Ahora al mostrar el dataframe, verás el contenido completo
display(complex_filter[['model', 'domain', 'generation']].head(3))

# Si quieres restaurar los valores por defecto de pandas:
pd.reset_option('display.max_colwidth')
pd.reset_option('display.max_rows')

,model,domain,generation
0,human,abstracts,"The recent advancements in artificial intelligence (AI) combined with the\nextensive amount of data generated by today's clinical systems, has led to the\ndevelopment of imaging AI solutions across the whole value chain of medical\nimaging, including image reconstruction, medical image segmentation,\nimage-based diagnosis and treatment planning. Notwithstanding the successes and\nfuture potential of AI in medical imaging, many stakeholders are concerned of\nthe potential risks and ethical implications of imaging AI solutions, which are\nperceived as complex, opaque, and difficult to comprehend, utilise, and trust\nin critical clinical applications. Despite these concerns and risks, there are\ncurrently no concrete guidelines and best practices for guiding future AI\ndevelopments in medical imaging towards increased trust, safety and adoption.\nTo bridge this gap, this paper introduces a careful selection of guiding\nprinciples drawn from the accumulated experiences, consensus, and best\npractices from five large European projects on AI in Health Imaging. These\nguiding principles are named FUTURE-AI and its building blocks consist of (i)\nFairness, (ii) Universality, (iii) Traceability, (iv) Usability, (v) Robustness\nand (vi) Explainability. In a step-by-step approach, these guidelines are\nfurther translated into a framework of concrete recommendations for specifying,\ndeveloping, evaluating, and deploying technically, clinically and ethically\ntrustworthy AI solutions into clinical practice."
1,human,abstracts,"High-quality training data play a key role in image segmentation tasks.\nUsually, pixel-level annotations are expensive, laborious and time-consuming\nfor the large volume of training data. To reduce labelling cost and improve\nsegmentation quality, interactive segmentation methods have been proposed,\nwhich provide the result with just a few clicks. However, their performance\ndoes not meet the requirements of practical segmentation tasks in terms of\nspeed and accuracy. In this work, we propose EdgeFlow, a novel architecture\nthat fully utilizes interactive information of user clicks with edge-guided\nflow. Our method achieves state-of-the-art performance without any\npost-processing or iterative optimization scheme. Comprehensive experiments on\nbenchmarks also demonstrate the superiority of our method. In addition, with\nthe proposed method, we develop an efficient interactive segmentation tool for\npractical data annotation tasks. The source code and tool is avaliable at\nhttps://github.com/PaddlePaddle/PaddleSeg."
2,human,abstracts,"The success of deep learning methods in medical image segmentation tasks\nheavily depends on a large amount of labeled data to supervise the training. On\nthe other hand, the annotation of biomedical images requires domain knowledge\nand can be laborious. Recently, contrastive learning has demonstrated great\npotential in learning latent representation of images even without any label.\nExisting works have explored its application to biomedical image segmentation\nwhere only a small portion of data is labeled, through a pre-training phase\nbased on self-supervised contrastive learning without using any labels followed\nby a supervised fine-tuning phase on the labeled portion of data only. In this\npaper, we establish that by including the limited label in formation in the\npre-training phase, it is possible to boost the performance of contrastive\nlearning. We propose a supervised local contrastive loss that leverages limited\npixel-wise annotation to force pixels with the same label to gather around in\nthe embedding space. Such loss needs pixel-wise computation which can be\nexpensive for large images, and we further propose two strategies, downsampling\nand block division, to address the issue. We evaluate our methods on two public\nbiomedical image datasets of different modalities. With different amounts of\nlabeled data, our methods c

## Uso de Stylometrix

### Test de extracción

In [22]:
# Get sample dataframe 'generation'
generation_sample = train_noadv_df[['id', 'model', 'domain', 'generation']].sample(n=5, random_state=3)
display(generation_sample)

,id,model,domain,generation
109610,7da3073c-8722-42d5-9415-b80e228cc9b3,gpt3,books,\n\nInvoluntary Witness is a novel by popular ...
280116,e13d2b2a-6668-4c9b-bd41-f81d93086fa5,mistral-chat,recipes,Ingredients:\n- 1 (16.5 oz) refrigerated sugar...
434720,f3bcdde8-76d7-497f-b74d-5578e01c1f46,mistral-chat,wiki,Relationship Development Intervention (RDI) is...
218000,715002c0-88b3-4899-a148-45dccbfbd827,mistral-chat,poetry,"In the office, where work is done\nWith deadli..."
32635,f42bbfb9-1d7f-4873-a3fa-f7b15f1037b2,gpt2,abstracts,We present a partially reversible U-Net for ef...


#### Extracción de rasgos en un registro

In [23]:
print(generation_sample['generation'].iloc[0])
outputList = split_text_into_sentences(generation_sample['generation'].iloc[0])



Involuntary Witness is a novel by popular Russian writer Sergey Lukyanenko, originally published in 2000.

The plot revolves around a young man named Kirill, who is drafted into the army against his will. He is sent to fight in a war against an unknown enemy, and is captured by the enemy. He is then forced to witness the execution of his friends and comrades, before he is himself killed.
[OK] Total de oraciones: 4


In [24]:
for sentence in outputList:
    print(sentence)
    print("-----")

Involuntary Witness is a novel by popular Russian writer Sergey Lukyanenko, originally published in 2000.
-----
The plot revolves around a young man named Kirill, who is drafted into the army against his will.
-----
He is sent to fight in a war against an unknown enemy, and is captured by the enemy.
-----
He is then forced to witness the execution of his friends and comrades, before he is himself killed.
-----


In [25]:
stylo = sm.StyloMetrix('en', debug=True, save_path="testVisual/single") # define langauge, one of ('de','en', 'pl', 'ru', 'ukr')
metrics = stylo.transform(outputList) # list of texts
metrics

c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_trf' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
  0%|          | 0/4 [00:00<?, ?it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
100%|██████████| 4/4 [00:00<00:00,  5.20it/s]

File saved in location: testVisual\single\sm_output1.csv
File saved in location: testVisual\single\sm_debug1.csv


(                                                text  POS_VERB  POS_NOUN  \
 0  Involuntary Witness is a novel by popular Russ...  0.133333  0.133333   
 1  The plot revolves around a young man named Kir...  0.222222  0.222222   
 2  He is sent to fight in a war against an unknow...  0.277778  0.166667   
 3  He is then forced to witness the execution of ...  0.277778  0.166667   
 
     POS_ADJ   POS_ADV   POS_DET  POS_INTJ  POS_CONJ  POS_PART   POS_NUM  ...  \
 0  0.200000  0.066667  0.066667       0.0  0.000000  0.000000  0.066667  ...   
 1  0.055556  0.000000  0.166667       0.0  0.000000  0.000000  0.000000  ...   
 2  0.055556  0.000000  0.166667       0.0  0.055556  0.055556  0.000000  ...   
 3  0.000000  0.055556  0.055556       0.0  0.111111  0.055556  0.000000  ...   
 
          RE  ASF  ASM   OM  RCI  DMC   OR       QAS   PA   PR  
 0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  
 1  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.111111  0.0  0.0  
 2  0.00000

#### Extracción de rasgos en múltiples registros

In [26]:
stylo = sm.StyloMetrix('en', debug=True, save_path="testVisual/multiple")

for idx, i in enumerate(generation_sample.index):
    print(f"Processing index: {i} (iteration {idx})")
    text = generation_sample['generation'].loc[i]
    sentences = split_text_into_sentences(text)
    
    # Cambiar los nombres de los archivos para cada iteración
    stylo.output_name = f"output_iter{idx}_idx{i}"
    stylo.debug_name = f"debug_iter{idx}_idx{i}"
    features = stylo.transform(sentences)
    print(f"Saved: {stylo.output_name}.csv and {stylo.debug_name}.csv")

    

c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_trf' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Processing index: 109610 (iteration 0)
[OK] Total de oraciones: 4


  0%|          | 0/4 [00:00<?, ?it/s]c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
100%|██████████| 4/4 [00:00<00:00,  4.19it/s]


File saved in location: testVisual\multiple\output_iter0_idx1096101.csv
File saved in location: testVisual\multiple\debug_iter0_idx1096101.csv
Saved: output_iter0_idx109610.csv and debug_iter0_idx109610.csv
Processing index: 280116 (iteration 1)
[OK] Total de oraciones: 17


100%|██████████| 17/17 [00:03<00:00,  4.59it/s]


...

File saved in location: testVisual\multiple\output_iter2_idx4347201.csv
File saved in location: testVisual\multiple\debug_iter2_idx4347201.csv
Saved: output_iter2_idx434720.csv and debug_iter2_idx434720.csv
Processing index: 218000 (iteration 3)
[OK] Total de oraciones: 1


100%|██████████| 1/1 [00:01<00:00,  1.04s/it]


File saved in location: testVisual\multiple\output_iter3_idx2180001.csv
File saved in location: testVisual\multiple\debug_iter3_idx2180001.csv
Saved: output_iter3_idx218000.csv and debug_iter3_idx218000.csv
Processing index: 32635 (iteration 4)
[OK] Total de oraciones: 5


100%|██████████| 5/5 [00:01<00:00,  3.49it/s]

File saved in location: testVisual\multiple\output_iter4_idx326351.csv
File saved in location: testVisual\multiple\debug_iter4_idx326351.csv
Saved: output_iter4_idx32635.csv and debug_iter4_idx32635.csv
